In [ ]:
!pip install numpy
!pip install matplotlib
!pip install scikit-image
!pip install pillow
!pip install opencv-python
!pip install timm
!pip install tqdm
# Check is at least python 3.9
import sys 
assert (sys.version_info.major == 3) and (sys.version_info.minor == 9)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from PIL import Image
from typing import Callable
import os
from datetime import datetime
from enum import Enum
import random
from skimage.color import rgb2gray, rgb2hsv
from skimage import filters
from skimage import segmentation
from skimage.morphology import remove_small_holes, remove_small_objects
import cv2 as cv

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import timm
import torchvision
from torchvision.models.vision_transformer import vit_b_16
from torchvision.models import ViT_B_16_Weights

In [ ]:
import platform

# Get os name
os_name = platform.system().lower()
num_workers = 8

# OS X
if 'darwin' in os_name:
    print("Detected OS X")
    %pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1
# Linux 
elif 'linux' in os_name:
    print("Detected Linux")
    %pip install torch==1.8.1 torchvision==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html
# Windows 
else:
    print("Detected Windows")
    num_workers = 0  # Hard fix for Windows users
    %pip install torch==1.12.0 torchvision==0.13 -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
class ImageType(Enum):
    NEUTRAL = "1. neutral_bg"
    NOISY = "2. noisy_bg"
    HAND = "3. hand"
    NEUTRAL_OOD = "4. neutral_bg_outliers"
    NOISY_OOD = "5. noisy_bg_outliers"
    HAND_OOD = "6. hand_outliers"
    
def loadImageOfType(type: ImageType, downsample: int = 1, grayscale: bool = False):
    # directory = f"./data/train/{type}"
    directory = os.path.join(".", "data", "train", type.value)
    images = os.listdir(directory)
    img = np.array(Image.open(os.path.join(directory, random.choice(images)))) 
    if downsample > 1:
        img = img[::downsample, ::downsample, :]
    if grayscale:
        img = (rgb2gray(img)*255).astype(np.uint8)
        
    # img = filters.sobel(img)
    
    return img
        


## Preprocessing

In [ ]:
def filter_circles(circle_array):
    # print(circle_array)
    # Convert the array to a NumPy array
    
    # Find unique center coordinates
    unique_centers, unique_indices = np.unique(circle_array[:, :2], axis=0, return_index=True)
    # print(unique_indices)
    
    # Get the indices of circles with largest radius for each unique center
    largest_circles = []
    for center in unique_centers:
      largest_circle = np.array((center[0], center[1], 0))
      for j in range(len(circle_array)):
        circle = circle_array[j]
        if (circle[0] == center[0] and circle[1]==center[1] and circle[2] > largest_circle[2]):
          largest_circle[2] = circle[2] 
      largest_circles.append(largest_circle)
  
    # print(largest_circles)
               
      

    # # Convert the list of indices to a NumPy array
    # max_radius_indices = np.array(max_radius_indices)
    # print(max_radius_indices)
    
    return np.array(largest_circles)

In [ ]:
# the input is a cv image so x and y are swapped
def extract_circles_images(x, y, r, img, down):
    factor = r
    left = (x - factor)*down
    right = (x + factor)*down
    up = (y - factor)*down
    bottom = (y + factor)*down
    output = img.copy()
    output[:, :, 0] = img[:, :, 2]
    output[:, :, 2] = img[:, :, 0]
    return output[up:bottom, left:right, :]

In [ ]:
def find_circles(path, debug = False):
    img = cv.imread(path)
    full_size_img = img.copy()
    down = 3
    size = (int(img.shape[1]/down), int(img.shape[0]/down))
    img = cv.resize(img, size, interpolation=cv.INTER_LINEAR)

    output = img.copy()
    output[:, :, 0] = img[:, :, 2]
    output[:, :, 2] = img[:, :, 0]
    img[:, :, 2] = 0

    gray = cv.blur(cv.cvtColor(img, cv.COLOR_BGR2GRAY), (8, 8))

    # detect circles in the image
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT_ALT, dp=1, minDist = 50, minRadius=50, maxRadius=200, param1=5, param2=0.6)
    # ensure at least some circles were found
    circles_imgs = []
    if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")
        circles = filter_circles(circles)
        print(f"detected {len(circles)} circles")
    
        for (x, y, r) in circles:
            # print(x, y, r)
            # draw the circle in the output image, then draw a rectangle
                # corresponding to the center of the circle
            # cv.circle(output, (x, y), r, (0, 255, 0), 4)
            # cv.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 255, 0), -1)
            new_circle = extract_circles_images(x, y, r, full_size_img, down)
            circles_imgs.append(new_circle)
            if debug:
                plt.imshow(new_circle)
                plt.show()
            # circles_imgs.append(extract_circles_images(x, y, r, output))
    
        # show the output image
    if debug:
        plt.imshow(output)
        plt.show()
    return circles_imgs

In [ ]:
def get_training_data_paths():
    training_paths = []
    base_dir = "./data/train/"
    for d in os.listdir(base_dir):
        training_dir = f"{base_dir}/{d}"
        for path in os.listdir(training_dir):
            file_path = f"{training_dir}/{path}"
            training_paths.append(file_path)
            # print(file_path)
    return training_paths
        
# get_training_data_paths()

In [ ]:
def get_label_for_image(path):
    filename = path.split("/")[-1].split(".")[0]

    data = np.genfromtxt('./data/train_labels.csv', delimiter=',', dtype=str)

# Assume the first column contains the strings you want to search through
    first_column = data[:, 0]

    # Find the index of the first entry that matches your string
    index = np.where(first_column == filename)[0]
    
    if index.size > 0:
        # print(f"The first entry matching '{filename}' is at index {index[0]}.")
        #we add 0 in order to account for non coins
        label_vector = np.append(data[index[0]][1:], 0)
        # print(label_vector)
        return torch.from_numpy(label_vector.astype(np.float32))
        # You can access the corresponding row using data[index[0]]
    else:
        # print(f"No entry matching '{filename}' found.")
        return torch.zeros(0)


In [ ]:
# d = "./data/train/1. neutral_bg//"
# d = "./data/train/2. noisy_bg///"
# d = "./data/train/3. hand///"
d = "./data/train/5. noisy_bg_outliers/"
# d = "./data/train/6. hand_outliers///"
# d = "./data/test//"
# for f in os.listdir(d):

paths = get_training_data_paths()
p = paths[0]
if get_label_for_image(p).shape[0] == 0:
    print("wrong filename")

circles = find_circles(paths[0], debug=True)
# for path in get_training_data_paths():
# img = cv.imread(f"{d}/{f}")[::3, ::3].astype(np.uint8)
# img[:, :, 1] = 0



In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

In [ ]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
resnet152 = torchvision.models.resnet152(weights=torchvision.models.ResNet152_Weights.IMAGENET1K_V2).to(device)
def extract_features(input_image):
    # or any of these variants
    # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
    # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
    # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
    # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
    resnet152.eval()
    feature_extractor = torch.nn.Sequential(*list(resnet152.children())[:-1])

    image_dim = torch.tensor([input_image.shape[0]]).to(device)
    # print(image_dim)

    input_image = Image.fromarray(input_image)
    # input_image.show()
    preprocess = transforms.Compose([
        # transforms.Resize(256),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available

    input_batch = input_batch.to(device)
    feature_extractor.to(device)

    with torch.no_grad():
        features = feature_extractor(input_batch)
    # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
    # print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    # probabilities = torch.nn.functional.softmax(output[0], dim=0)
    features = torch.flatten(features)
    features = torch.cat((features, image_dim))

    # print(features.size())
    return features




In [ ]:
# # Step 1: Load a pre-trained ViT model
# vit = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1).to(device)
# # model = timm.create_model('vit_base_patch16_224', pretrained=True,  ViT_B_16_Weights.IMAGENET1K_V1:)

# # Step 2: Modify the model to be a feature extractor
# # Remove the classification head
# vit.head = nn.Identity()
# def extract_features_vit(input_image):

#     # Set the model to evaluation mode
#     vit.eval()

#     # Step 3: Define the transformation pipeline
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
#         # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
#     ])

#     # Step 4: Load an image and preprocess it
#     # image_path = get_training_data_paths()[0]
#     # image_path = 'path_to_your_image.jpg'  # Replace with your image path
#     image = Image.fromarray(input_image)
    
#     # image = Image.open(image_path)
#     image = transform(image).to(device)
#     # display_image = (image.permute(1, 2, 0)*255).int()
#     # print(display_image)
#     # plt.imshow(display_image)
#     # plt.show()
    
#     image = image.unsqueeze(0)  # Add batch dimension
#     feats = vit._process_input(image)
#     # Step 5: Extract features
#     with torch.no_grad():
#         # features = vit(image)
#         batch_class_token = vit.class_token.expand(image.shape[0], -1, -1)
#         feats = torch.cat([batch_class_token, feats], dim=1)

#         feats = vit.encoder(feats)

#         # We're only interested in the representation of the CLS token that we appended at position 0
#         feats = feats[:, 0]

#         # print(feats.shape)
#         return feats


In [ ]:
for circle in circles:
    print(extract_features(circle).shape)

In [ ]:
def get_test_data_paths():
    test_paths = []
    base_dir = "./data/test/"
    for path in os.listdir(base_dir):
        file_path = f"{base_dir}/{path}"
        test_paths.append(file_path)
            # print(file_path)
    return test_paths



## Training on individual coin

In [ ]:
linear = torch.nn.Sequential( 
    torch.nn.Linear(in_features = 2049, out_features = 512), 
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 512, out_features = 17), 
    # torch.nn.ReLU(),
    # torch.nn.Linear(in_features = 256, out_features = 17), 
    # torch.nn.Softmax(dim=1) 
)

linear = linear.to(device)


In [ ]:
class_names = ["chf_5", "chf_2", "chf_1", "chf_0.5", "chf_0.2", "chf_0.1", "chf_0.05", "eur_2", "eur_1", "eur_0.5", "eur_0.2", "eur_0.1", "eur_0.05", "eur_0.02", "eur_0.01", "OOD", "bg"]


In [ ]:
def get_training_set_size():
    size = 0
    for index, c in enumerate(class_names):
        labeled_coins_dir = "./data/labeled_coins/"
        selected_dir = f"{labeled_coins_dir}{c}"
        files = os.listdir(selected_dir)
        size += len(files)
    return size

print(get_training_set_size())

In [ ]:
import random
from PIL import Image as im 

criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(linear.parameters(), lr=5e-4)

data_set_size = get_training_set_size()

trainX = torch.zeros((data_set_size, 2049)).to(device)
trainY = torch.zeros(data_set_size).to(device)

data_index = 0



for index, c in enumerate(class_names):
    print(f"extracting class {index}")
    labeled_coins_dir = "./data/labeled_coins/"
    selected_dir = f"{labeled_coins_dir}{c}"
    files = os.listdir(selected_dir)
    for j, file in enumerate(files):
        target = torch.tensor([index]).to(device)

        selected_file_path = f"{selected_dir}/{file}"
        coin = im.open(selected_file_path)
        # degrees = random.randint(0, 360)
        # coin = coin.rotate(degrees, resample=Image.BICUBIC)
    #     # coin.show()
        
    #     """
    #     extracting features + training
    #     """

    #     optimizer.zero_grad()
        x = extract_features(np.array(coin))
        trainX[data_index] = x
        trainY[data_index] = index

        
        data_index += 1

nb_epochs = 1000

for epoch in range(nb_epochs):
    optimizer.zero_grad()
            
    pred = linear(trainX)
    # print(pred.shape)
    loss = criterion(pred, trainY.to(device).long()) 
    loss.backward() 
    # print(f"loss at epoch {epoch} - {loss}")
    # if epoch %1 == 0:
    print(f"loss at {epoch} - {loss}")
    optimizer.step() 
        
            



    #     x = torch.unsqueeze(x, 0)
        
    #     selected_file = random.choice(files)

#update after each sample
# for i in range(5000):
    
#     """
#     Selecting image at random
#     """
#     index = np.random.randint(0, 17)
#     class_nb = len(class_names)
#     # target = torch.zeros(class_nb).to(device)
#     # target[index] = 1
#     target = torch.tensor([index]).to(device)
#     # target[0] = index
    

#     labeled_coins_dir = "./data/labeled_coins/"
#     selected_dir = f"{labeled_coins_dir}{class_names[index]}"
#     files = os.listdir(selected_dir)
#     selected_file = random.choice(files)
#     selected_file_path = f"{selected_dir}/{selected_file}"
#     # print(selected_file_path)
    
#     coin = im.open(selected_file_path)
#     degrees = random.randint(0, 360)
#     coin = coin.rotate(degrees, resample=Image.BICUBIC)
#     # coin.show()
    
#     """
#     extracting features + training
#     """

#     optimizer.zero_grad()
#     x = extract_features_vit(np.array(coin))
#     # x = torch.unsqueeze(x, 0)

#     pred = linear(x)
#     # print(pred)
#     # print(target)
#     loss = criterion(pred, target) 
#     loss.backward() 
#     if i %10 == 0:
#         print(f"loss at {i} - {loss}")
#     optimizer.step() 



In [ ]:
print(trainX.shape)
print(trainY)


In [ ]:
sum(p.numel() for p in model.parameters())

In [ ]:
csv_header = "id,5CHF,2CHF,1CHF,0.5CHF,0.2CHF,0.1CHF,0.05CHF,2EUR,1EUR,0.5EUR,0.2EUR,0.1EUR,0.05EUR,0.02EUR,0.01EUR,OOD"



In [ ]:
def path_to_filename(path):
    return path.split("/")[-1].split(".")[0]

## Testing

In [ ]:
index = np.random.randint(0, 17)
csv_file = csv_header
csv_file += "\n"
with torch.no_grad():
# index = 16
    linear.eval()

    # labeled_coins_dir = "./data/labeled_coins/"
    # selected_dir = f"{labeled_coins_dir}{class_names[index]}"
    # files = os.listdir(selected_dir)
    # selected_file = random.choice(files)
    # selected_file_path = f"{selected_dir}/{selected_file}"
    # print(selected_file_path)

    for i, path in enumerate(get_test_data_paths()):
        print(i)
    
        # path = "./data/test/L0000113.JPG"
        coins = find_circles(path)
        # coins = find_circles("./data/train/1. neutral_bg/L1010277.JPG")
        
        p = np.zeros(16, dtype=np.uint8)
        for coin in coins:
            shape = coin.shape
            if shape[0] <10 or shape[1]<10: continue
            # print(coin.shape)
        
            
            
            # coin = np.array(im.open(selected_file_path))
            # coin.show()


            x = extract_features(coin).to(device)
            # print(x)
            x = torch.unsqueeze(x, 0)

            pred_test = linear(x).flatten()
            # print(pred_test)
            pred_class = torch.argmax(pred_test)
            if pred_class == 16: continue
            p[pred_class] += 1
        # print(f"pred {pred_class}")
        # print(f"target: {index}")
        
        # print(p)
        p_string = np.array2string(p, separator=', ', precision=0).replace("[", "").replace("]", "")
        csv_row = f"{path_to_filename(path)},{p_string}"
        # print(csv_row)
        csv_file += csv_row
        csv_file += "\n"

with open('./data/submission.csv', 'w') as file:
    # Write a string to the file
    file.write(csv_file)
    
# print(csv_row)


## extracting all coins

In [ ]:
from PIL import Image as im 
paths = get_training_data_paths()
print(len(paths))

# for path in paths:
for path in paths:
    circles =find_circles(path)
    for i, circle in enumerate(circles):
        file_number = path.split("/")[-1].split(".")[0]
        print(file_number)
        im.fromarray(circle).save(f"./data/unlabeled_coins/{file_number}_{i}.jpg")
    


### label hd coins

In [ ]:
target_dir = "./data/hd_coins/"
source_dir = "./data/labeled_coins/"
unlabeled_dir = "./data/unlabeled_coins/"
for d in os.listdir(source_dir):
    class_dir = target_dir + d
    if not os.path.exists(class_dir):
        os.mkdir(class_dir)
    
    for file in os.listdir(source_dir+d):
        src_file = unlabeled_dir+file
        dst_file = target_dir + d+"/"+file
        os.rename(src_file, dst_file)
        # print(dst_file)